# LangChainのAgent機能をテストする

In [ ]:
%pip install openai
%pip install langchain

In [1]:
import os

# ホームディレクトリの下にあるファイルのパスを作成
file_path = os.path.join(os.path.expanduser("~"), 'dotsecret')

# ファイルを開いて各行を読みます
variables = {}
with open(file_path, 'r') as file:
    for line in file:
        # 空白行を無視します
        if line.strip():
            key, value = line.strip().split('=')
            os.environ[key] = value

In [2]:
from langchain.chat_models import ChatOpenAI   
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)  

from langchain.agents import tool  
@tool   
def get_word_length(word: str) -> int:
    """単語の長さを計算する関数"""
    return len(word)

tools = [get_word_length]  


In [3]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a very powerful assistant..."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [4]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# エージェントの組み立て
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm
    | OpenAIFunctionsAgentOutputParser()
)


In [5]:
from langchain.agents import AgentExecutor   

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  

agent_executor.invoke({"input": "how many letters in the word 123456789012334?"})  




> Entering new AgentExecutor chain...
There are 15 letters in the word "123456789012334".

> Finished chain.


{'input': 'how many letters in the word 123456789012334?',
 'output': 'There are 15 letters in the word "123456789012334".'}